## Storing data in a networked Postgres database

To avoid the "pg_config executable not found" error when installing the psycopg2 package, ensure that the necessary PostgreSQL libraries and headers are installed on your system. For Debian/Ubuntu systems, execute sudo apt-get install libpq-dev python3-dev. For Red Hat/CentOS/Fedora, use sudo yum install postgresql-devel python3-devel. On macOS, install PostgreSQL with Homebrew using brew install postgresql. Alternatively, you can install psycopg2-binary with pip install psycopg2-binary, which includes the required dependencies without needing compilation. However, psycopg2-binary is more suitable for development and limited testing rather than production environments. If issues persist after these steps, try reinstalling SQLAlchemy with pip install --force-reinstall sqlalchemy. Following these steps will enable smooth installation and operation of psycopg2 and SQLAlchemy for connecting to a PostgreSQL database.

In [ ]:
!pip install sqlalchemy psycopg2

In [1]:
import exchange_calendars as xcals
import pandas as pd
from IPython.display import Markdown, display
from openbb import obb
from sqlalchemy import create_engine, text
from sqlalchemy.exc import ProgrammingError

In [2]:
obb.user.preferences.output_type = "dataframe"

Database connection parameters

In [ ]:
username = ""
password = ""
host = "127.0.0.1"
port = "5432"
database = "market_data"

In [ ]:
DATABASE_URL = f"postgresql://{username}:{password}@{host}:{port}/postgres"
base_engine = create_engine(DATABASE_URL)

Function to create a new database and return an engine for that database

In [ ]:
def create_database_and_get_engine(db_name, base_engine):
    conn = base_engine.connect()
    conn = conn.execution_options(isolation_level="AUTOCOMMIT")

    try:
        conn.execute(text(f"CREATE DATABASE {db_name};"))
    except ProgrammingError:
        pass
    finally:
        conn.close()

    conn_str = base_engine.url.set(database=db_name)

    return create_engine(conn_str)

Create the database and get an engine for it

In [ ]:
engine = create_database_and_get_engine("stock_data", base_engine)

Function to fetch historical stock data for a given symbol and date range, and add a 'symbol' column

In [ ]:
def get_stock_data(symbol, start_date=None, end_date=None):
    data = obb.equity.price.historical(
        symbol,
        start_date=start_date,
        end_date=end_date,
        provider="yfinance",
    )
    data.reset_index(inplace=True)
    data["symbol"] = symbol
    return data

Function to save the fetched stock data to a PostgreSQL database

In [ ]:
def save_data_range(symbol, engine, start_date=None, end_date=None):
    data = get_stock_data(symbol, start_date, end_date)
    data.to_sql("stock_data", engine, if_exists="append", index=False)

Function to save the stock data for the last trading session to a PostgreSQL database

In [ ]:
def save_last_trading_session(symbol, engine):
    today = pd.Timestamp.today()
    data = get_stock_data(symbol, today, today)
    data.to_sql("stock_data", engine, if_exists="append", index=False)

Save data for multiple stock symbols in the specified date range

In [ ]:
for symbol in ["SPY", "QQQ", "DIA"]:
    save_data_range(
        symbol, engine=engine, start_date="2020-06-01", end_date="2023-01-01"
    )

Read and display data for the stock symbol "SPY" from the database

In [ ]:
df_1 = pd.read_sql_query("SELECT * from stock_data where symbol='SPY'", engine)
display(df_1)

Read and display data for "SPY" where the volume is greater than 100,000,000

In [ ]:
df_2 = pd.read_sql_query(
    "SELECT * from stock_data where symbol='SPY' and volume > 100000000", engine
)
display(df_2)

The best way to build your database is to first download history. You do this once to “backfill” the historic data. Then you schedule the script to run every trading day after the market close to capture that day’s data going forward.

There are a lot of references to schedule a Python job on Mac and Windows so I won’t cover it in detail here.

Here are two that will work:

​Schedule on Mac/Linux​ : https://theautomatic.net/2020/11/18/how-to-schedule-a-python-script-on-a-mac/
​Schedule on Windows​ : https://www.jcchouinard.com/python-automation-using-task-scheduler/

**Jason Strimpel** is the founder of <a href='https://pyquantnews.com/'>PyQuant News</a> and co-founder of <a href='https://www.tradeblotter.io/'>Trade Blotter</a>. His career in algorithmic trading spans 20+ years. He previously traded for a Chicago-based hedge fund, was a risk manager at JPMorgan, and managed production risk technology for an energy derivatives trading firm in London. In Singapore, he served as APAC CIO for an agricultural trading firm and built the data science team for a global metals trading firm. Jason holds degrees in Finance and Economics and a Master's in Quantitative Finance from the Illinois Institute of Technology. His career spans America, Europe, and Asia. He shares his expertise through the <a href='https://pyquantnews.com/subscribe-to-the-pyquant-newsletter/'>PyQuant Newsletter</a>, social media, and has taught over 1,000+ algorithmic trading with Python in his popular course **<a href='https://gettingstartedwithpythonforquantfinance.com/'>Getting Started With Python for Quant Finance</a>**. All code is for educational purposes only. Nothing provided here is financial advise. Use at your own risk.